<a href="https://colab.research.google.com/github/Gaurav-phatkare/Retrieval-Augmented-Generation_RAG/blob/main/RAG_Fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Jul 25 10:32:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import torch

## building

1. Open a PDF document
2. Format the text of that PDF suitable for embedding model
3. embed into chunks and then convert it into numerical representation
4. use vector db to search relevent cuunk
5. crete prompt that incoporates the retrived peices of text
6. generate answer

## Import PDF


In [1]:
import os
import requests

pdf_path = "human-nutrition-text.pdf"

# download PDF

if not os.path.exists(pdf_path):
  print("File doesnt exists")

  url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

  filename = pdf_path
  res = requests.get(url)

  if res.status_code == 200:
    with open(filename, "wb") as file:
      file.write(res.content)
    print(f"file has been downloaded and stored as {filename}")

  else:
    print(f"failed to download file, Status code : {res.status_code}")
else:
  print("file exists")

File doesnt exists
file has been downloaded and stored as human-nutrition-text.pdf


In [3]:
!pip install pyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 57.6 MB/s eta 0:00:00


In [4]:
!pip install tqdm

In [6]:
import fitz
from tqdm.auto import tqdm


def text_formatter(text:str):
  cleaned_text = text.replace("\n", " ").strip()
  return cleaned_text


def open_and_read_pdf(pdf_path):
  doc = fitz.open(pdf_path)

  pages_and_texts = []

  for page_number, page in tqdm(enumerate(doc)):
    text = page.get_text()

    text = text_formatter(text=text)

    pages_and_texts.append({"page_number" : page_number - 41,
                            "page_char_count" : len(text),
                            "page_word_count" : len(text.split(" ")),
                            "page_sentence_count_raw" : len(text.split(". ")),
                            "page_token_count": len(text) / 4,
                            "text" : text})

  return pages_and_texts


pages_and_texts = open_and_read_pdf(pdf_path)
pages_and_texts[:3]




0it [00:00, ?it/s]

[{'page_number': -41,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count_raw': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -40,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''},
 {'page_number': -39,
  'page_char_count': 320,
  'page_word_count': 54,
  'page_sentence_count_raw': 1,
  'page_token_count': 80.0,
  'text': 'Human Nutrition: 2020  Edition  UNIVERSITY OF HAWAI‘I AT MĀNOA  FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM  ALAN TITCHENAL, SKYLAR HARA,  NOEMI ARCEO CAACBAY, WILLIAM  MEINKE-LAU, YA-YUN YANG, MARIE  KAINOA FIALKOWSKI REVILLA,  JENNIFER DRAPER, GEMADY  LANGFELDER, CHERYL GIBBY, CHYNA  NICOLE CHUN, AND ALLISON  CALABRESE'}]

In [7]:
import random

random.sample(pages_and_texts, 2)

[{'page_number': 609,
  'page_char_count': 297,
  'page_word_count': 60,
  'page_sentence_count_raw': 2,
  'page_token_count': 74.25,
  'text': 'Choy Sum by  pxhere.com  / CCO  Introduction  UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM  He pūko`a kani `āina  A coral reef strengthens into land.  Learning Objectives  By the end of this chapter you will be able to:  Introduction  |  609'},
 {'page_number': 555,
  'page_char_count': 1603,
  'page_word_count': 281,
  'page_sentence_count_raw': 15,
  'page_token_count': 400.75,
  'text': 'routinely to prevent colds. Contrary to this popular practice,  however, there is no good evidence that vitamin C prevents a cold.  A review of more than fifty years of studies published in 2004 in the  Cochrane Database of Systematic Reviews concluded that taking  vitamin C routinely does not prevent colds in most people, but it  does slightly reduce cold severity and duration. Moreover, taking  megad

In [8]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition
1,-40,0,1,1,0.00,
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...
4,-37,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...


In [9]:
df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000
mean,562.50000,1148.594371,198.889901,9.972682,287.148593
std,348.86387,560.441673,95.747365,6.187226,140.110418
min,-41.00000,0.000000,1.000000,1.000000,0.000000
25%,260.75000,762.750000,134.000000,4.000000,190.687500
50%,562.50000,1232.500000,215.000000,10.000000,308.125000
75%,864.25000,1605.250000,271.250000,14.000000,401.312500
max,1166.00000,2308.000000,429.000000,32.000000,577.000000
